In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from transformers import AutoModel, AutoTokenizer
from typing import List, Tuple

from sae_model import SparseAutoencoder
from activation_utils import get_llm_activations_residual_stream
from sae_trainer import extract_activations, create_data_loader, train_sparse_autoencoder

/Users/itsukikuwahara/codes/research/sae/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [23]:
# extract_activationsの変数設定
llm_model_name = "distilgpt2"    # 使用するLLMモデル名    
texts = [
    "The quick brown fox jumps over the lazy dog.",
    "A large language model can process and generate text.",
    "Cats enjoy sleeping in warm, sunny spots.",
    "This is a sample text for testing the Sparse Autoencoder.",
    "PyTorch is a widely used deep learning framework." ,
    ]  # 使用するテキスト
target_layer_idx = 5    # 抽出するLLMの層インデックス
num_samples_for_training = 5  # 訓練に使用するサンプル数

# create_data_loaderの変数設定
batch_size = 256  # バッチサイズ

# train_sparse_autoencoderの変数設定
num_epochs = 200  # 訓練エポック数
sae_l1_coeff = 1e-4 # スパース性の度合いを調整する係数

In [24]:
tokenizer = AutoTokenizer.from_pretrained(llm_model_name)
llm_model = AutoModel.from_pretrained(llm_model_name)
training_texts = [texts[i] for i in range(num_samples_for_training)]

activations, activations_dict = get_llm_activations_residual_stream(
    llm_model, tokenizer, training_texts, target_layer_idx
)

In [ ]:
llm_model.

In [25]:
data_loader = create_data_loader(activations, batch_size)
sae_model, training_losses, resonctruction_losses, sparsity_losses, sae_feature_dim, input_dim = train_sparse_autoencoder(
    activations, data_loader, num_epochs, sae_l1_coeff
)

Using device: mps
Starting SAE training for 200 epochs...
Epoch 1/200, Total Loss: 31.9262, Recon Loss: 31.9260, Sparse Loss: 1.2784
Epoch 2/200, Total Loss: 35.1297, Recon Loss: 35.1296, Sparse Loss: 1.2702
Epoch 3/200, Total Loss: 24.1889, Recon Loss: 24.1888, Sparse Loss: 1.2112
Epoch 4/200, Total Loss: 21.4916, Recon Loss: 21.4914, Sparse Loss: 1.1893
Epoch 5/200, Total Loss: 21.4065, Recon Loss: 21.4063, Sparse Loss: 1.1921
Epoch 6/200, Total Loss: 17.6462, Recon Loss: 17.6461, Sparse Loss: 1.2009
Epoch 7/200, Total Loss: 12.7755, Recon Loss: 12.7754, Sparse Loss: 1.2211
Epoch 8/200, Total Loss: 10.5396, Recon Loss: 10.5395, Sparse Loss: 1.2589
Epoch 9/200, Total Loss: 10.5166, Recon Loss: 10.5165, Sparse Loss: 1.3118
Epoch 10/200, Total Loss: 8.7684, Recon Loss: 8.7683, Sparse Loss: 1.3706
Epoch 11/200, Total Loss: 5.5191, Recon Loss: 5.5189, Sparse Loss: 1.4325
Epoch 12/200, Total Loss: 4.0787, Recon Loss: 4.0786, Sparse Loss: 1.4992
Epoch 13/200, Total Loss: 4.6639, Recon Loss:

In [26]:
activations_dict["This is a sample text for testing the Sparse Autoencoder."]

tensor([[-2.6997e-03,  3.6206e-01,  3.2446e-02,  ..., -1.9162e-01,
          1.3057e-01, -8.9368e-02],
        [-7.5613e-02, -9.1808e-04, -4.8235e-01,  ...,  2.3693e-01,
          2.7107e-01,  2.7540e-01],
        [-2.7496e-01,  2.8702e-01,  2.3608e-01,  ..., -1.4405e-01,
         -5.7160e-02,  2.1048e-01],
        ...,
        [-2.1572e+00,  1.5088e+00,  8.2650e-01,  ...,  1.0375e-01,
         -6.5441e-01,  1.6608e+00],
        [-9.8130e-02, -9.6860e-02,  4.0309e-01,  ...,  2.5921e-02,
         -1.7225e-01, -1.8101e-01],
        [ 5.4393e-01, -3.5867e-01, -3.1157e-01,  ..., -8.3079e-02,
         -1.0296e-01, -1.8767e-02]], device='mps:0')

In [27]:
device = torch.device("mps" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
sae_model.to(device)
# sae_model.eval()

Using device: cpu


SparseAutoencoder(
  (encoder): Linear(in_features=768, out_features=3072, bias=True)
  (relu): ReLU()
  (decoder): Linear(in_features=3072, out_features=768, bias=True)
)

In [28]:
token_info_list = []
all_sae_features_list = []
global_token_idx = 0

# トークナイザーにpad_tokenが設定されているか確認 (活性化抽出時と条件を合わせるため)
if tokenizer.pad_token is None:
    if tokenizer.eos_token is not None:
        tokenizer.pad_token = tokenizer.eos_token
        print(f"トークナイザーのpad_tokenをeos_token ({tokenizer.pad_token}) に設定しました。")
    else:
        # これはデモスクリプト (demo_train_sae_gpt.py, demo_sae_train.py) や
        # activation_utils.py 内の処理と整合性を取る必要があります。
        print("警告: トークナイザーにpad_tokenが設定されていません。'[PAD]'を追加します。")
        tokenizer.add_special_tokens({'pad_token': '[PAD]'})
        # LLMモデルの埋め込み層のリサイズが必要な場合がある点に注意 (model.resize_token_embeddings(len(tokenizer)))

if not activations_dict:
    print("活性化辞書 (activations_dict) が空です。特徴分析をスキップします。")
else:
   for original_text, token_activations in activations_dict.items():
      # 活性ベクトルをGPUに転送
      token_activations = token_activations.to(device)
      
      # 学習済みのSAEモデルを使用して、トークンの活性化をエンコード
      with torch.no_grad():
         sae_model_pre_relu = sae_model.encoder(token_activations)
         sae_features_for_text = sae_model.relu(sae_model_pre_relu)

      all_sae_features_list.append(sae_features_for_text.cpu())

      # トークンを取得
      inputs = tokenizer(original_text, return_tensors="pt", padding="max_length", truncation=True, max_length=128)
      
      # パディングトークンを除外するためにAttention Maskを使用
      attention_mask = inputs["attention_mask"].squeeze(0)
      input_ids_squeeze = inputs["input_ids"].squeeze(0)

      # トークンIDを取得
      actual_tokens_ids_for_text = input_ids_squeeze[attention_mask == 1]
      actual_tokens_str_list = tokenizer.convert_ids_to_tokens(actual_tokens_ids_for_text)

      if len(actual_tokens_str_list) != sae_features_for_text.shape[0]:
         print(f"Warning: Mismatch in token count for text: {original_text}")
         continue
      
      for token_idx_in_text in range(sae_features_for_text.shape[0]):
         token_info_list.append({
            "original_text": original_text,
            "token_idx_in_text": token_idx_in_text,   # テキスト内の実施あのトークンに対するインデックス
            "token_str": actual_tokens_str_list[token_idx_in_text],  # トークンの文字列表現
            "global_token_idx": global_token_idx,  # データセット全体を通したトークンのインデックス
         })
         
         global_token_idx += 1
         
# 以下結果の表示
if not all_sae_features_list:
   print("No SAE features found for the token.")

else:
   concatenated_sae_features = torch.cat(all_sae_features_list, dim=0)  # Shape: (num_tokens, sae_feature_dim)
   sae_total_features = concatenated_sae_features.shape[0]
   
   num_sae_features_to_analyze = min(10, sae_total_features) # 最大10個のSAE特徴を分析
   num_top_tokens_per_feature = 5  # 各SAE特徴に対して上位5つのトークンを分析

   # 指定した数のSAE特徴を分析するためのループ
   for feature_idx_to_analyze in range(num_sae_features_to_analyze):
      # 現在のSAE特徴次元に対応する前トークンの活性を取得
      feature_column_activation = concatenated_sae_features[:, feature_idx_to_analyze]
      
      # 上位k個の活性化とそのグローバルインデックスを取得
      actual_k = min(num_top_tokens_per_feature, len(feature_column_activation))
      if actual_k == 0 : continue
      
      top_k_values, top_k_global_indices = torch.topk(feature_column_activation, k=actual_k)
      
      print(f"\n--- SAE Feature {feature_idx_to_analyze} を最も強く活性化するトークン")
      
      if top_k_values.numel() == 0:
         print("No top tokens found for this feature.")
         continue
      
      # 上位k個のトークンの情報を表示
      for rank, (activation_value, global_token_idx_item) in enumerate(zip(top_k_values, top_k_global_indices)):
         global_idx = global_token_idx_item.item()    # テンソルから値を取り出す
         if global_idx < len(token_info_list):        
            token_info = token_info_list[global_idx]  # 取得したトークン情報
            
            text_snippet = token_info["original_text"]   # 元のテキスト
            
            # 文脈表示のために、元のテキストを再度トークナイズ(表示用)
            inputs_ctx = tokenizer(text_snippet,
                                 return_tensors="pt",
                                 truncation=True,
                                 max_length=128,
                                 padding="max_length",
                                 return_attention_mask=True)
            ids_ctx = inputs_ctx["input_ids"].squeeze()[inputs_ctx["attention_mask"].squeeze() == 1]
            tokens_ctx = tokenizer.convert_ids_to_tokens(ids_ctx)
            
            # 上記 tokens_ctx リスト内でのインデックスに相当
            tok_idx_in_ctx = token_info["token_idx_in_text"]
            
            context_window_size = 3    # 表示する前後のトークン数
            start_idx = max(0, tok_idx_in_ctx - context_window_size)
            end_idx = min(len(tokens_ctx), tok_idx_in_ctx + context_window_size + 1)
            
            context_display_parts = []                  
            for i in range(start_idx, end_idx):
               if i == tok_idx_in_ctx:
                  context_display_parts.append(f"**{tokens_ctx[i]}**")
               else:
                  context_display_parts.append(tokens_ctx[i])
            context_str = " ".join(context_display_parts)

            print(f"  順位 {rank + 1}: 活性化値 = {activation_value.item():.4f}")
            print(f"    トークン: '{token_info['token_str']}' (テキスト内の実トークンindex: {tok_idx_in_ctx})")
            print(f"    文脈: {context_str}")
            text_preview = (text_snippet[:70] + '...') if len(text_snippet) > 70 else text_snippet # テキストのプレビュー
            print(f"    元テキスト (一部): \"{text_preview}\"")
         else:
            print(f"  順位 {rank + 1}: エラー - グローバルインデックス {global_idx} が範囲外です。")                  
            


--- SAE Feature 0 を最も強く活性化するトークン
  順位 1: 活性化値 = 0.0000
    トークン: 'The' (テキスト内の実トークンindex: 0)
    文脈: **The** Ġquick Ġbrown Ġfox
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 2: 活性化値 = 0.0000
    トークン: 'Ġquick' (テキスト内の実トークンindex: 1)
    文脈: The **Ġquick** Ġbrown Ġfox Ġjumps
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 3: 活性化値 = 0.0000
    トークン: 'Ġbrown' (テキスト内の実トークンindex: 2)
    文脈: The Ġquick **Ġbrown** Ġfox Ġjumps Ġover
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 4: 活性化値 = 0.0000
    トークン: 'Ġfox' (テキスト内の実トークンindex: 3)
    文脈: The Ġquick Ġbrown **Ġfox** Ġjumps Ġover Ġthe
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."
  順位 5: 活性化値 = 0.0000
    トークン: 'Ġjumps' (テキスト内の実トークンindex: 4)
    文脈: Ġquick Ġbrown Ġfox **Ġjumps** Ġover Ġthe Ġlazy
    元テキスト (一部): "The quick brown fox jumps over the lazy dog."

--- SAE Feature 1 を最も強く活性化するトークン
  順位 1: 活性化値 = 0.0000
    トークン: 'The' (テキスト内の実トークンindex: 0)
    文脈: **The*